In [15]:
import scipy
import pandas as pd

from implicit.als import AlternatingLeastSquares
from implicit.evaluation import mean_average_precision_at_k

In [68]:
ratings = pd.read_csv("./data/ml-100k/u.data", sep="\t", header=None)
ratings.columns = ['user_id', 'item_id', 'rating', 'timestamp']
ratings.sort_values('timestamp', inplace=True)
ratings['score'] = (ratings['rating'] > 2).apply(int)

In [70]:
ratings[ratings['score'] == 0]

,user_id,item_id,rating,timestamp,score
97222,851,687,2,874728168,0
85905,712,510,2,874729749,0
85933,712,42,1,874729935,0
71838,712,234,2,874729935,0
51048,712,177,2,874730155,0
...,...,...,...,...,...
78878,683,325,2,893286346,0
80614,729,338,1,893286373,0
64971,729,901,1,893286491,0
55267,729,683,2,893286511,0


In [71]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(ratings, test_size=0.2, shuffle=False)

In [72]:
train_pivot = pd.pivot_table(
    train,
    index="user_id", 
    columns="item_id", 
    values="score"
)
test_pivot = pd.pivot_table(
    test,
    index="user_id", 
    columns="item_id", 
    values="score"
)

print(train_pivot.shape)
print(test_pivot.shape)

(751, 1616)
(301, 1448)


In [73]:
shell = pd.pivot_table(
    ratings, 
    index="user_id", 
    columns="item_id", 
    values="score", 
    aggfunc=lambda x: 0
)
shell.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
train_pivot = shell + train_pivot
test_pivot = shell + test_pivot

train_pivot = (train_pivot + 1).fillna(0)
test_pivot = (test_pivot + 1).fillna(0)
print(train_pivot.shape)
print(test_pivot.shape)
## (943, 1682)
## (943, 1682)

train_pivot.head()


(943, 1682)
(943, 1682)


item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,2.0,2.0,2.0,2.0,0.0,2.0,2.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
train_pivot_sparse = scipy.sparse.csr_matrix(train_pivot.values)
test_pivot_sparse = scipy.sparse.csr_matrix(test_pivot.values)

In [23]:
model = AlternatingLeastSquares(factors=10, random_state=42)
model.fit(train_pivot_sparse)

  0%|          | 0/15 [00:00<?, ?it/s]

In [24]:
import numpy as np
unique_items = np.array(train_pivot.columns)
user_id = 14
recomendations_ids, scores = model.recommend(user_id, train_pivot_sparse[user_id])
recomendations = unique_items[recomendations_ids]

In [25]:
print('Recomendations ids: {}'.format(recomendations_ids))
print('Recomendations for user {}: {}'.format(user_id, recomendations))

Recomendations ids: [293 116  99 275 244 287 283 150 125 596]
Recomendations for user 14: [294 117 100 276 245 288 284 151 126 597]


In [26]:
map_at10 = mean_average_precision_at_k(model, train_pivot_sparse, test_pivot_sparse, K=10)
print('Mean Average Precision at 10: {:.3f}'.format(map_at10))

  0%|          | 0/301 [00:00<?, ?it/s]

Mean Average Precision at 10: 0.087


#   факторизационными машинами. 

In [27]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

In [28]:
model = LightFM(no_components=10, loss='logistic', random_state=42)
model.fit(train_pivot_sparse, epochs=30)

In [67]:
item_ids = np.arange(0, train_pivot_sparse.shape[1])
list_pred = model.predict(user_id, item_ids)
recomendations_ids = np.argsort(-list_pred)[:10]
recomendations = unique_items[recomendations_ids]
recomendations

array([ 50, 294, 258, 100, 181, 288, 286,   1, 300, 121], dtype=int64)

In [61]:
item_ids = np.arange(0, train_pivot_sparse.shape[1])
list_pred = model.predict(user_id, item_ids)
recomendations_ids = np.argsort(-list_pred)[:10]
recomendations = unique_items[recomendations_ids]
print('Recomendations for user {}: {}'.format(user_id, recomendations))

Recomendations for user 14: [ 50 294 258 100 181 288 286   1 300 121]


In [30]:
map_at10 = precision_at_k(model, test_pivot_sparse, k=10).mean()
print('Mean Average Precision at 10: {:.2f}'.format(map_at10))
## Mean Average Precision at 10: 0.32

Mean Average Precision at 10: 0.32


In [56]:
x = np.array([4, 0, 3, 5])

In [57]:
np.sort(x)

array([0, 3, 4, 5])

In [58]:
np.argsort(x)

array([1, 2, 0, 3], dtype=int64)